<h1><center>Анализ тональности текста</center></h1>

## Считывание датасета

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_excel('doc_comment_summary.xlsx', sheet_name=0, header=None)
dataset.columns = ('text', 'evaluation')

In [3]:
dataset.head()

,text,evaluation
0,Но при мужчине ни одна приличная женщина не по...,-1
1,Украина это часть Руси искусственно отделенная...,-1
2,Как можно говорить об относительно небольшой к...,-1
3,1.2014. а что они со своими поляками сделали?...,0
4,у а фильмы... Зрители любят диковинное. у ме...,0


## Предобработка датасета

Удаляем все "нейтральные" комментарии.

In [4]:
dataset = dataset[dataset['evaluation'] != 0].reset_index(drop=True)

Почему-то не все комментарии являются строкой, как мы можем видеть.

In [5]:
dataset['text'].apply(type).unique()

array([<class 'str'>, <class 'int'>, <class 'float'>], dtype=object)

Исправляем это.

In [6]:
text_col_wrong_types = dataset['text'].apply(lambda x: isinstance(x, int) or isinstance(x, float))
dataset = dataset[~text_col_wrong_types].reset_index(drop=True)

In [7]:
dataset.head()

,text,evaluation
0,Но при мужчине ни одна приличная женщина не по...,-1
1,Украина это часть Руси искусственно отделенная...,-1
2,Как можно говорить об относительно небольшой к...,-1
3,Государство не может сейчас платить больше и м...,-1
4,эксплуатируемые способны только на бунты - бес...,-1


Так как я использовал алгоритмы бинарной классификации, а не регрессии, то нужно сократить количество классов до двух. В данном случае это просто: ставим "негативный" класс 0, если оценка меньше 0, иначе 1.

In [8]:
dataset['evaluation'] = np.where(dataset['evaluation'] < 0, 0, 1)

## Проблема дисбаланса классов

In [9]:
from imblearn.under_sampling import RandomUnderSampler

Как мы можем видеть, негативных комментариев в разы больше положительных (классика). От этого необходимо избавиться.

In [10]:
dataset['evaluation'].value_counts()

0    10735
1     2182
Name: evaluation, dtype: int64

In [11]:
sampler = RandomUnderSampler(random_state=42)
dataset, _ = sampler.fit_resample(dataset, dataset['evaluation'])

Просто удаляем случайным образом негативные комментарии, и проблема решена

In [12]:
dataset['evaluation'].value_counts()

0    2182
1    2182
Name: evaluation, dtype: int64

## Предобработка текста

Я решил оставить только русские буквы и считать это слово, обрамленное проблемами, словом.

In [13]:
def preprocess_text(text):
    text = re.sub(r'[^а-яА-Я]', ' ', text)
    text = re.sub(r'\ {2,}', ' ', text)
    text = text.strip().lower()
    return text

Мы можем видеть пример работы моей функции обработки текста.

In [14]:
preprocess_text('..  1.Что ЭТО?!2.Зачем (с какой целью, для чего, за каким ...) читать Сорокина?  Не в том дело, что не надо больше, а в том что  Куда уже дальше? .  Возможно ли теперь  Больше ада ? По моему это предел, человеческая фантазия не в силах вообразить себе нечто большее (хотя и такое себе человек не вообразит). Меня на ночь клонит в метафизику - было ли это Великое зачеловеческое откровение или оно ещё только впереди? В любом случае гордитесь Россией мы на пороге великого метафизического прорыва в нечеловеческие сферы.   Норвежские культисты... Ктулху Фтагн !!!P.S. Чего только не придумает съехавшая с катушек баба... ради денег и пиара.  Дельсаль. Берсерк. Массакр')

'что это зачем с какой целью для чего за каким читать сорокина не в том дело что не надо больше а в том что куда уже дальше возможно ли теперь больше ада по моему это предел человеческая фантазия не в силах вообразить себе нечто большее хотя и такое себе человек не вообразит меня на ночь клонит в метафизику было ли это великое зачеловеческое откровение или оно ещ только впереди в любом случае гордитесь россией мы на пороге великого метафизического прорыва в нечеловеческие сферы норвежские культисты ктулху фтагн чего только не придумает съехавшая с катушек баба ради денег и пиара дельсаль берсерк массакр'

Применим функцию к датасету.

In [15]:
dataset['text'] = dataset['text'].apply(preprocess_text)
dataset['text'].head()

0    многие владельцы ипотек оказались раззоренными...
1    другое дело что желающих иметь именно промышле...
2    бстановка что надо послевкусие дайте две поско...
3    то то никак крым себя не прокормит все дотации...
4    ранее и штрафовали и народные дружинники по ул...
Name: text, dtype: object

## Разделение выборки

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
docs_train, docs_test, evals_train, evals_test = train_test_split(dataset['text'], dataset['evaluation'], train_size=0.8, random_state=42)

## Векторизация слов в тексте

В качестве метода сегментезции текста был использован метод **TF-IDF**, а в качестве алгоритма обработки слов был использован **стемминг**.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize, download   
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

In [19]:
stemmer = SnowballStemmer(language='russian')

Пример работы стемминга

In [138]:
stemmer.stem('аграрный')

'аграрн'

Функция токенизации (разбиения текста на слова).

In [21]:
def tokenize(text):
    tokens = text.split()
    stems = [stemmer.stem(item) for item in tokens if item not in stopwords.words('russian')]
    return stems

In [22]:
vectorizer = TfidfVectorizer(tokenizer=tokenize)

In [23]:
vectors_arr = vectorizer.fit_transform(docs_train)

Как мы можем видеть, у нас 32777 слов (колонок в таблице), что очень много и грустно.

In [24]:
vectors_arr.shape

(3491, 32777)

Пример стеммизированных слов

In [25]:
vectorizer.get_feature_names()[30:40]

['аборт',
 'абортивн',
 'абортмахер',
 'абр',
 'абрам',
 'абрамов',
 'абрамович',
 'абрамовн',
 'абрамс',
 'абрамыч']

In [26]:
vectors = pd.DataFrame(data=vectors_arr.todense(), columns=vectorizer.get_feature_names())
vectors

,,а,аа,ааа,аааа,ааааа,аааааа,аааааааа,аааааааааа,ааааааааааа,...,ячейк,яш,яшин,яшк,ящета,ящик,ящита,ящичек,ящэ,яяяя
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


32777 колонок. Ужас.

## Построение моделей классификации

In [39]:
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [35]:
def assess_model(model, y_true):
    model_evals = model.predict(vectorizer.transform(docs_test).todense())
    return confusion_matrix(evals_test, model_evals)

### Наивный байесовский классификатор

In [32]:
gnd = GaussianNB()
gnb.fit(vectors, evals_train)

GaussianNB()

In [36]:
assess_model(gnb, evals_test)

array([[309, 160],
       [127, 277]], dtype=int64)

### SVM

In [29]:
svc = SVC(cache_size=600, class_weight='balanced')
svc.fit(vectors, evals_train)

SVC()

In [37]:
assess_model(svc, evals_test)

array([[386,  83],
       [127, 277]], dtype=int64)

### Decision Tree

In [40]:
dtc = DecisionTreeClassifier()
dtc.fit(vectors, evals_train)

DecisionTreeClassifier()

In [41]:
assess_model(dtc, evals_test)

array([[320, 149],
       [139, 265]], dtype=int64)

## Подход на основе тонального словаря

In [18]:
import random
from pymorphy2 import MorphAnalyzer

Считывание данных.

In [19]:
words_eval = pd.read_excel('full word_rating_after_coding.xlsx', sheet_name=0, header=None)
words_eval.columns = ('word', 'evaluation')

Так как каждое слово представлено в таблице, как правило, несколькими оценками, которые могут разниться, я принял решение суммировать все оценки для каждого слова.

In [20]:
words_eval = words_eval.groupby('word').sum('evaluation')
words_eval

,evaluation
word,
абажур,-1
абориген,-2
аборт,-3
абортивный,-1
абсолютный,0
...,...
ясность,1
ясный,1
яхта,0


In [50]:
words_eval.iloc[[np.argmax(words_eval.values)]]

,evaluation
word,
волшебный,10


In [53]:
words_eval[words_eval['evaluation'] == 10]

,evaluation
word,
волшебный,10
доброта,10


Сделаем из текста список слов

In [21]:
docs_words = docs_test.apply(lambda text: text.split())
docs_words

1760    [головный, кодекс, когда, неверие, неверующих,...
51      [нова, пошла, на, сговор, и, дала, добро, на, ...
4308    [блюющие, на, пасху, христиане, полагаю, все, ...
2292    [путин, всего, достиг, сам, и, без, чье, либо,...
1044    [васяню, можно, ругать, как, хошь, а, вот, дру...
                              ...                        
3113    [де, просто, статистика, беда, скоро, кто, про...
1398    [многих, остались, принципиальные, вопросы, не...
3184    [у, меня, вопрос, совсем, глупый, почему, л, с...
809     [бедные, они, то, ли, дело, в, целом, поддержи...
1659    [все, беды, от, бап, да, какой, в, жопу, верто...
Name: text, Length: 873, dtype: object

Преобразуем таблицу в словарь

In [22]:
tone_dict = dict([(k, v) for k, v in zip(words_eval.index, words_eval['evaluation'])])

Применяем **лемматизацию** для изменения слов. Сильно влияет на результат (в положительную сторону, конечно).

In [25]:
morph = MorphAnalyzer()
docs_words = docs_words.apply(lambda words: [morph.parse(word)[0].normal_form for word in words if word not in stopwords.words('russian')])
docs_words

1760    [головный, кодекс, неверие, неверующий, станов...
51      [новый, пойти, сговор, дать, добро, агрессия, ...
4308    [блевать, пасха, христианин, полагать, метафор...
2292    [путин, достигнуть, чей, либо, помощь, такой, ...
1044    [васянить, ругать, хотеть, другой, низзить, ди...
                              ...                        
3113    [де, просто, статистика, беда, скоро, проезжат...
1398    [многий, остаться, принципиальный, вопрос, физ...
3184    [вопрос, глупый, почему, л, расположение, одет...
809     [бедный, дело, целое, поддерживать, ответить, ...
1659    [беда, бап, жопа, вертолетоносец, война, франц...
Name: text, Length: 873, dtype: object

Получаем список оценок для каждого комментария

In [26]:
tone_dict_score = docs_words.apply(lambda text: sum(map(lambda word: tone_dict.get(word, 0), text)))
tone_dict_score

1760   -10
51     -29
4308    -6
2292    17
1044   -51
        ..
3113   -20
1398     7
3184    15
809    -38
1659   -26
Name: text, Length: 873, dtype: int64

In [30]:
print(tone_dict_score.max())
print(tone_dict_score.min())

78
-167


Бинаризуем их

In [132]:
conditions  = [tone_dict_score < 0, tone_dict_score > 0, tone_dict_score == 0]
choices     = [0, 1, random.randint(0, 1)]
tone_dict_eval = np.select(conditions, choices)
tone_dict_eval[:10]

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 1])

Результат хуже SVM. Очень странно

In [133]:
confusion_matrix(evals_test, tone_dict_eval)

array([[389,  80],
       [171, 233]], dtype=int64)